Converting pip cells

In [ ]:
%horus convert

In [ ]:
pip install --upgrade pip

In [ ]:
# Install requirements
!pip install -r Requirements.txt 

In [ ]:
# View AWS Configuration
!aws configure set default.region us-east-1
!aws configure list


Restart kernel to load the new libraries.

----Code Start----

Import statements

In [ ]:
# Imports 
import os
from ray.util import inspect_serializability
import ray
import pyarrow.fs as pq
import pandas as pd

# Making use of datetime for dates, works for every day of the year (30,31,28 day problems go away)
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

Connect to ray cluster

In [ ]:
from ray.util.client import ray as rayclient
if rayclient.is_connected():
    ray.util.disconnect()

ray.util.connect('{ray_head}:10001'.format(ray_head=os.environ['RAY_CLUSTER']))

Reusable definitions here

In [ ]:
# Reusable definitions here
bucketName = 'DH-SECURE-THANOS-RAY-USE'
endpoint = 'https://s3.upshift.redhat.com'

prefixPathRead = 'raydev'
prefixPathWrite = 'raydev-write-demo'
metricName = 'cluster_version'

AWS_ACCESS_KEY = %env AWS_ACCESS_KEY_ID
AWS_SECRET_KEY = %env AWS_SECRET_ACCESS_KEY

year = '2021'
month = '01'
day = '01'

# Read path
read_path = f'{prefixPathRead}/metric={metricName}'

# Write path
write_path = f'{prefixPathWrite}/metric={metricName}'


In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(access_key=AWS_ACCESS_KEY, secret_key=AWS_SECRET_KEY, endpoint_override=endpoint)

Following cells reads one days worth of parquet files and writes back a single parquet file
Month 1, Day 1.
1. Fails currently because worker nodes keep dying.
2. Success! Successful compaction for a days worth of parquets. Next is a month.

In [ ]:
# Read path
read_path = f'{prefixPathRead}/metric={metricName}'
read_bucket_uri = f's3://{bucketName}/{read_path}/year={year}/month={month}/day={day}'

# Write path
write_path = f'{prefixPathWrite}/metric={metricName}'
write_bucket_uri = f's3://{bucketName}/{write_path}/year={year}/month={month}/day={day}'

In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(access_key=AWS_ACCESS_KEY, secret_key=AWS_SECRET_KEY, endpoint_override=endpoint)
# Reading parquet using Ray through filesystem
df = ray.data.read_parquet(paths=read_bucket_uri, filesystem=fs_pyarrow)

# Writing back a single parquet file
df.repartition(1).write_parquet(path=write_bucket_uri, filesystem=fs_pyarrow)

Following cells try compaction for a whole month:
1. Reading from multiple directories seems to be not supported
2. Union of dataframes is supported, can be done before the compaction step. https://github.com/ray-project/ray/issues/24598
Status: Fails due to memory limits, cannot load a dataframe for more than several days worth of parquets

In [ ]:
# Read path
read_path = f'{prefixPathRead}/metric={metricName}'
read_bucket_uri = f's3://{bucketName}/{read_path}/year={year}/month={month}/day={day}'
# Holds paths 
read_bucket_multiple = []

# Creating paths for every day in the month.
for i in range(2,32):
    read_bucket_multiple.append(f"s3://{bucketName}/{read_path}/year={year}/month={month}/day="f"{i:02d}")
                           
#print(read_bucket_multiple)


# Write path
write_path = f"{prefixPathWrite}/metric={metricName}"
write_bucket_uri = f's3://{bucketName}/{write_path}/year={year}/month={month}'


In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(endpoint_override=endpoint)


# Initial read - First day to start a dataframe
master_df = ray.data.read_parquet(paths=f's3://{bucketName}/{prefixPathRead}/metric={metricName}/year={year}/month={month}/day=01', filesystem=fs_pyarrow)

# Reading remaining days
for x in read_bucket_multiple:
    df = ray.data.read_parquet(paths=x, filesystem=fs_pyarrow) # Reading parquets for each day using Ray through filesystem
    master_df = master_df.union(df) # Followed by unioning the dataframe each time
    


In [ ]:
# Writing back a single parquet file
df.repartition(1).write_parquet(path=write_bucket_uri, filesystem=fs_pyarrow)

Making use of datetime to run compaction for every day, union dataframes and finally writeback

In [ ]:
firstDay = date(2021, 1, 1)
lastDay = firstDay + relativedelta(months = 1)
print(lastDay, firstDay)
duration = lastDay - firstDay
print(duration)
for _ in range(12):
    for i in range(duration.days):
        day = firstDay + timedelta(days = i)
        # Set year, month and and day here
        year, month, day = (day.year, day.month, day.day)
        currentPath = f's3://{bucketName}/{read_path}/year={year}/month='f'{month:02d}''/day='f'{day:02d}'
        print(currentPath)
        # Code here:
        # <Read dataframe>
        df = ray.data.read_parquet(paths=currentPath, filesystem=fs_pyarrow)
        # <Union dataframe> if not day 1.
        if day != 1:
            master_df = master_df.union(df) # Followed by unioning the dataframe each time
        else:
            df = master_df

    # Code here:
    # <Writeback single parquet for the month>
    df.repartition(1).write_parquet(path=f's3://{bucketName}/{write_path}/year={year}/month='f'{month:02d}', filesystem=fs_pyarrow)
    firstDay = lastDay
    lastDay = firstDay + relativedelta(months = 1)


Compact everyday and store as one parquet (365 files)

In [ ]:
#%%capture output 
firstDay = date(2021,2,1) #February onwards
lastDay = firstDay + relativedelta(months = 1)
#print(lastDay, firstDay)
duration = lastDay - firstDay
#print(duration)
for _ in range(12):
    for i in range(duration.days):
        day = firstDay + timedelta(days = i)
        # Set year, month and and day here
        year, month, day = (day.year, day.month, day.day)        
        # Code here:
        # <Read dataframe>
        currentReadPath = f's3://{bucketName}/{read_path}/year={year}/month='f'{month:02d}''/day='f'{day:02d}'
        print("Reading from:",currentReadPath)
        df = ray.data.read_parquet(paths=currentReadPath, filesystem=fs_pyarrow)
        # <Writeback single parquet for the day>
        currentWritePath = f's3://{bucketName}/{write_path}/year={year}/month='f'{month:02d}''/day='f'{day:02d}'
        print("Writing to:", currentWritePath)
        df.repartition(1).write_parquet(path=currentWritePath, filesystem=fs_pyarrow)
    firstDay = lastDay
    lastDay = firstDay + relativedelta(months = 1)
    duration = lastDay

Trying to see if Ray remote functions make a difference

In [ ]:
#%%capture output 
@ray.remote
def compaction():
    firstDay = date(2021,1,1)
    lastDay = firstDay + relativedelta(months = 1)
    #print(lastDay, firstDay)
    duration = lastDay - firstDay
    #print(duration)
    InitializeS3()
    for _ in range(12):
        for i in range(duration.days):
            day = firstDay + timedelta(days = i)
            # Set year, month and and day here
            year, month, day = (day.year, day.month, day.day)
            currentReadPath = f's3://{bucketName}/{read_path}/year={year}/month='f'{month:02d}''/day='f'{day:02d}'
            print("Reading from:",currentReadPath)
            # Code here:
            # <Read dataframe>
            df = ray.data.read_parquet(paths=currentReadPath, filesystem=fs_pyarrow)
            # <Writeback single parquet for the day>
            currentWritePath = f's3://{bucketName}/{write_path}/year={year}/month='f'{month:02d}''/day='f'{day:02d}'
            df.repartition(1).write_parquet(path=currentWritePath, filesystem=fs_pyarrow)
            print("Writing to:", currentWritePath)
        firstDay = lastDay
        lastDay = firstDay + relativedelta(months = 1)
        duration = lastDay

In [ ]:
compaction.remote()

Display cached output

In [ ]:
output.show()

Breaking down the code for read and write

In [ ]:
#Read one days worth
currentReadPath = f's3://{bucketName}/{read_path}/year=2021/month=01/day=04'
print("Reading from:",currentReadPath)
# Code here:
# <Read dataframe>
df = ray.data.read_parquet(paths=currentReadPath, filesystem=fs_pyarrow)


In [ ]:
print(df)

In [ ]:
df.repartition(1)

In [ ]:
#%%capture output 
currentWritePath = f's3://{bucketName}/{write_path}/year=2021/month=01/day=04'
print("Writing to:", currentWritePath)
df.repartition(1).write_parquet(path=currentWritePath, filesystem=fs_pyarrow)